# Creating Predictions
If you have some HSI images and want to get predictions from one of our pretrained models (or your own models), then we covered you in this notebook. Predictions can be computed based on a folder with HSI images and the `htc inference` command.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np

from htc import Config, LabelMapping, decompress_file, settings

[WARNING][dotenv.main] Python-dotenv could not parse statement starting at line 6                             main.py:28

In [2]:
data_dir = settings.data_dirs.test_dataset28may
output_dir = settings.results_dir/ "predictions"
data_dir
settings.results_dir
data_dir
print(settings._external)

None


> Note: If you want to use your own data and it does not fit into the default structure (e.g. because you have non-Tivita images), they you need to write your [own DataPath class](./CustomDataPath.md) first.

In [4]:
!env
!htc inference --input-dir $data_dir/subjects/P086/2021_04_15_09_22_02 --output-dir $output_dir --model image --run-folder 2022-02-03_22-58-44_generated_default_model_comparison
assert _exit_code == 0, "Inference was not successful"  # noqa: F821.................................................................................................

SHELL=/bin/bash
PYTHONUNBUFFERED=1
WSL2_GUI_APPS_ENABLED=1
CONDA_EXE=/home/lucas/miniconda3/bin/conda
_CE_M=
WSL_DISTRO_NAME=Ubuntu
WT_SESSION=f6adf399-41d2-4515-b707-e5a7b91a6e34
ELECTRON_RUN_AS_NODE=1
HOMEBREW_PREFIX=/home/linuxbrew/.linuxbrew
VSCODE_AMD_ENTRYPOINT=vs/workbench/api/node/extensionHostProcess
PATH_Tivita_HeiPorSPECTRAL_example=~/dkfz/htc/tests/test_28may/HeiPorSPECTRAL_example:shortcut=test_dataset28may
NAME=Code
PWD=/home/lucas/dkfz/htc/tutorials
LOGNAME=lucas
CONDA_ROOT=/home/lucas/miniconda3
CONDA_PREFIX=/home/lucas/miniconda3/envs/htc-dev
MANPATH=/home/linuxbrew/.linuxbrew/share/man:
PYDEVD_IPYTHON_COMPATIBLE_DEBUGGING=1
_=/usr/bin/env
PLOTLY_RENDERER=plotly_mimetype+notebook_connected
MOTD_SHOWN=update-motd
HOME=/home/lucas
LANG=C.UTF-8
WSL_INTEROP=/run/WSL/599_interop
LS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.a

This command searches for all HSI images in the given input directory, computes a prediction using the specified trained model (will also be downloaded if not available) and stores the result in the given output directory. You can use any of the pretrained models here.

In this example case, there is only one output image:

In [5]:
sorted(output_dir.rglob("*"))

[PosixPath('/home/lucas/dkfz/htc/tests/test_28may/test_results28may/predictions/image'),
 PosixPath('/home/lucas/dkfz/htc/tests/test_28may/test_results28may/predictions/image/2022-02-03_22-58-44_generated_default_model_comparison'),
 PosixPath('/home/lucas/dkfz/htc/tests/test_28may/test_results28may/predictions/image/2022-02-03_22-58-44_generated_default_model_comparison/0202-00118#2021_04_15_09_22_02.blosc'),
 PosixPath('/home/lucas/dkfz/htc/tests/test_28may/test_results28may/predictions/image/2022-02-03_22-58-44_generated_default_model_comparison/0202-00118#2021_04_15_09_22_02.html'),
 PosixPath('/home/lucas/dkfz/htc/tests/test_28may/test_results28may/predictions/image/2022-02-03_22-58-44_generated_default_model_comparison/config.json')]

Per default, this includes the predicted labels (stored in the blosc file), a visualization of the prediction (HTML file) and the config which was used for computing the predictions (which again is based on the config of the trained model). You can open the HTML file directly with any browser. The labels can be read with the `decompress_file()` function:

In [6]:
labels = decompress_file(sorted(output_dir.rglob("*.blosc"))[0])
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

The config can, for example, be used to recover the original label names:

In [7]:
config = Config(sorted(output_dir.rglob("config.json"))[0])
mapping = LabelMapping.from_config(config)

for l, c in zip(*np.unique(labels, return_counts=True)):
    print(f"{mapping.index_to_name(l)}: {c} pixels")

background: 307200 pixels


> Note: if you need the softmax values instead of the label indices of the prediction, add the `--predictions-type softmax` switch to `htc inference`. However, be aware that this requires much more disk space (around 17 MiB instead of 2 KiB per image)